In [1]:
!pwd

/content


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [3]:
!kaggle datasets download -d surajghuwalewala/ham1000-segmentation-and-classification

 99% 2.57G/2.59G [00:25<00:00, 165MB/s]
100% 2.59G/2.59G [00:25<00:00, 109MB/s]


In [ ]:
!unzip /content/ham1000-segmentation-and-classification.zip

In [5]:
import os
import numpy as np 
import pandas as pd
import tensorflow as tf

from PIL import Image

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [16]:
def img_np_convert(dataframe, image, h, w):
  df = pd.read_csv(dataframe)
  df['image'] = df['image'] +'.jpg'

  labels=['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
  label_list=[]
  for index, row in df.iterrows():
      temp = list(row)
      del temp[0]
      index=np.argmax(temp)
      label=labels[index]
      label_list.append(label)
  df['label']= label_list
  df=df.drop(labels, axis=1)

  sdir=image # main directory where data is stored
  save_dir=r'./' # output directory where model will be saved
  height=h  # image height
  width=w   # image width
  channels=3  # number of coloor channels
  batch_size=40  # model batch size for training and evaluation
  img_shape=(height, width, channels)
  img_size=(height, width)

  gen=ImageDataGenerator(rescale=1./255) 

  df_gen=gen.flow_from_dataframe( df, sdir, x_col='image', y_col='label', target_size=img_size, class_mode='categorical',
                                      color_mode='rgb', shuffle=True, batch_size=batch_size)
  images, labels = next(df_gen)
  for x in range(250):
    temp_images, temp_labels = next(df_gen)
    images = np.concatenate((images, temp_images), axis=0)
    labels = np.concatenate((labels, temp_labels), axis=0)
  return images, labels

In [17]:
df_path = '../content/GroundTruth.csv'
image_path = r'../content/images'
images, labels = img_np_convert(df_path, image_path, 32, 32)

Found 10015 validated image filenames belonging to 7 classes.


In [18]:
images.shape

(10015, 32, 32, 3)

In [19]:
labels.shape

(10015, 7)

In [ ]:
np.save('images96_96', images)

In [ ]:
np.save('labels96_96', labels)